### Задача:  
Разработать модель предсказания в каждом регионе объема продаж в штуках любого, выбранного вами, вида банки напитка Coca Cola.  
Модель должна предсказывать объем продаж на каждый час следующих суток (от последней даты транзакции в датасете).

In [799]:
import pandas as pd
import numpy as np
from sktime.forecasting.base import ForecastingHorizon
#from sktime.forecasting.naive import NaiveForecaster
from sktime.forecasting.arima import AutoARIMA
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")

In [800]:
data = pd.read_csv('dataset.csv', index_col=0)
display(data.sample(10, random_state=13))
print('Размерность датасета: ', data.shape)

,item_id,receipt_id,device_id,local_date,barcode,price,quantity,region
475282,11725120973,7401055919,352401547525226,2021-02-06 13:10:40,4600084388665,24.000000,1.000,Пермский край
2102122,11682540295,7377118917,352407076335888,2021-02-03 13:20:17,2000000000107,246.990005,0.442,Краснодарский край
1531729,11734646590,7406344901,352407930590845,2021-02-06 20:50:06,4606419005870,130.000000,1.000,Московская область
757038,11710050361,7392724566,352406575284803,2021-02-05 11:43:54,NaN,210.000000,1.000,Ростовская область
1147232,11732205129,7404993403,352403527442486,2021-02-06 16:50:32,NaN,5.000000,1.000,Новгородская область
686105,11726121192,7401605407,352406278254393,2021-02-06 11:53:06,4810344073020,1000.000000,1.000,Курская область
1574081,11667479891,7368510518,352406834320179,2021-02-02 12:47:07,NaN,90.000000,2.500,Краснодарский край
1248843,11685634611,7378885163,352399695785458,2021-02-03 20:01:01,4607031100011,51.000000,4.000,Республика Хакасия
802190,11730759337,7404185245,352402951220721,2021-02-06 15:26:00,4601653033276,105.000000,1.000,Рязанская область
290780,11714092766,7394966065,352406559653493,2021-02-05 14:33:52,4607087930181,7.200000,30.000,Ивановская область


Размерность датасета:  (2246094, 8)


Описание полей в датасете:  
item_id - Id транзакции  
receipt_id - Id чека  
device_id - Id кассового аппарата  
local_date - Дата и время покупки  
barcode - Штрихкод  
price - Цена покупки, рубли  
quantity - Количество купленного товара, шт.  
region - Регион

Проверим произвольно выбранные штихкоды на сайтах:  
https://ean-online.ru/  и https://service-online.su/text/shtrih-kod/  
4606419005870 - соответствует товару "Сливочное масло Экомилк традиционное 82,5 % 180 г бзмж"  
4607031100011 - соответствует товару "Пиво абаканское аян 0.5л 4.8% ст/б"  
Т.о. наш датасет содержит информацию о покупках различных продуктов, и нам необходимо выбрать только те записи, которые соответствуют покупке напитка Coca-Cola.

После небольшого исследования, приходим к выводу, что штрих код 5449000000286 соответствует товару "Напиток coca-cola безалкогольн газ 2л пэт", который является наиболее покупаемым и содержится в наибольшем количестве записей нашего датасета.

Так же были найдены и проверены штрихкоды:  
* 5740700995774 - Напиток coca-cola черри сильногаз жб 330мл - 0 записей
* 5449000017673 - Напиток coca-cola сильногаз 0.5л пл/б - 0 записей
* 54491472 - Напиток газированный coca-cola, 0.5л пластиковая бутылка - 489 записей
* 5449000054227 - Напиток coca-cola безалкогольн газ 1л пэт - 101 запись
* 54490147 - Напиток coca-cola 1l - 0 записей
* 5449000000996 - Напиток газированный coca-cola, 0.33л, жб - 526 записей
* 5449000009067 - Напиток coca-cola 2л - 0 записей
* 5449000000286 - Напиток coca-cola безалкогольн газ 2л пэт - 712 записей
* 5449000131836 - Напиток coca-cola zero безалкогольн газ 0.5л пэт - 22 записи

In [801]:
# Формируем датасет, содержащий информацию только о покупках Кока-колы
mask = data.barcode == '5449000000286' 
df_cola = data[mask]
df_cola.reset_index(drop=True, inplace=True)

### Проведем разведывательный анализ данных

In [802]:
# Выведем 10 случайных строк датафрейма, для предварительного осмотра данных 
display(df_cola.sample(10))
print('Размерность датасета: ', df_cola.shape)

,item_id,receipt_id,device_id,local_date,barcode,price,quantity,region
340,11695030658,7384245533,352405988717774,2021-02-04 11:43:58,5449000000286,125.0,1.0,Московская область
471,11702890786,7388690447,352404330688983,2021-02-04 18:26:49,5449000000286,119.0,1.0,Московская область
378,11733907436,7405940503,352407983664268,2021-02-06 19:04:08,5449000000286,95.0,1.0,Московская область
102,11729768550,7403633145,352408069722359,2021-02-06 18:34:45,5449000000286,142.0,1.0,Новосибирская область
231,11715170293,7395568347,352406036226376,2021-02-05 20:22:42,5449000000286,139.0,1.0,Республика Бурятия
657,11668972046,7369366916,352407766159560,2021-02-02 13:58:57,5449000000286,120.0,1.0,Краснодарский край
645,11688046364,7380262815,352407766159560,2021-02-03 18:20:02,5449000000286,120.0,1.0,Краснодарский край
637,11687998888,7380235490,352401779039216,2021-02-03 18:16:28,5449000000286,130.0,1.0,Московская область
145,11679188860,7375260115,352401133356508,2021-02-03 12:44:34,5449000000286,157.0,1.0,Тюменская область
381,11685965429,7379074969,352404673008115,2021-02-03 18:19:33,5449000000286,110.0,1.0,Пермский край


Размерность датасета:  (712, 8)


In [803]:
df_cola.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 712 entries, 0 to 711
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   item_id     712 non-null    int64  
 1   receipt_id  712 non-null    int64  
 2   device_id   712 non-null    int64  
 3   local_date  712 non-null    object 
 4   barcode     712 non-null    object 
 5   price       712 non-null    float64
 6   quantity    712 non-null    float64
 7   region      712 non-null    object 
dtypes: float64(2), int64(3), object(3)
memory usage: 44.6+ KB


In [804]:
display(df_cola.describe())
display(df_cola.describe(include='object'))

,item_id,receipt_id,device_id,price,quantity
count,7.120000e+02,7.120000e+02,7.120000e+02,712.000000,712.000000
mean,1.169810e+10,7.385838e+09,3.524574e+14,123.423455,1.155899
std,2.688573e+07,1.520228e+07,4.744213e+11,21.740303,0.558397
min,1.164707e+10,7.356689e+09,3.523991e+14,16.000000,1.000000
25%,1.167485e+10,7.372771e+09,3.524013e+14,110.000000,1.000000
50%,1.170241e+10,7.388417e+09,3.524041e+14,120.000000,1.000000
75%,1.172078e+10,7.398659e+09,3.524064e+14,135.000000,1.000000
max,1.173505e+10,7.406566e+09,3.566535e+14,200.000000,7.000000


,local_date,barcode,region
count,712,712,712
unique,712,1,53
top,2021-02-02 00:16:27,5449000000286,Республика Саха (Якутия)
freq,1,712,104


Записи датасета не содержат пропуски, а числовые признаки "price" и "quantity" не имеют отрицательных значений и выглядят реалистично. 
Датасет содержит данные о покупках из 53 регионов.  
Признак "item_id" (id транзакции) содержит только уникальные значения. При этом, он не является для нас информативным, его можно удалить.

In [805]:
# Удаляем признак "item_id"
df_cola.drop(['item_id'], axis=1, inplace=True)

Проверим записи на дубликаты

In [806]:
print('Количество дублирующихся записей: ', data.duplicated().sum())

Количество дублирующихся записей:  0


In [807]:
# Приведем признак "local_date" к типу datetime
df_cola.local_date = pd.to_datetime(df_cola.local_date).dt.to_period('H')
df_cola.head()

,receipt_id,device_id,local_date,barcode,price,quantity,region
0,7364791809,352402250963288,2021-02-02 00:00,5449000000286,140.0,1.0,Нижегородская область
1,7356688842,352402250963288,2021-02-01 00:00,5449000000286,140.0,1.0,Нижегородская область
2,7364799989,352402250963288,2021-02-02 00:00,5449000000286,140.0,1.0,Нижегородская область
3,7356692611,352406370923721,2021-02-01 00:00,5449000000286,120.0,1.0,Санкт-Петербург
4,7377491747,352399667615722,2021-02-03 13:00,5449000000286,110.0,1.0,Московская область


In [808]:
print('Самая ранняя дата в датасете:', df_cola.local_date.min())
print('Самая поздняя дата в датасете:', df_cola.local_date.max())

Самая ранняя дата в датасете: 2021-02-01 00:00
Самая поздняя дата в датасете: 2021-02-07 00:00


In [809]:
# Теперь можно удалить признаки receipt_id, device_id, barcode
# Так же удаляем признак price, - его можно было бы использовать для нахождения цены за единицу товара, но сейчас в этом нет необходимости
df_cola.drop(['receipt_id', 'device_id', 'barcode', 'price'], axis=1, inplace=True)
df_cola.head()

,local_date,quantity,region
0,2021-02-02 00:00,1.0,Нижегородская область
1,2021-02-01 00:00,1.0,Нижегородская область
2,2021-02-02 00:00,1.0,Нижегородская область
3,2021-02-01 00:00,1.0,Санкт-Петербург
4,2021-02-03 13:00,1.0,Московская область


Т.к. наш датасет содержит информацию о покупке только в определенные периоды времени, то нам необходимо заполнитьнедостающие периоды указав quantity = 0. Напишем для этого функцию

In [810]:
list_region = df_cola.region.unique()
for i in list_region:
    mask = df_cola.region == i
    tmp = df_cola[mask]

    delta=(tmp.local_date.max()-tmp.local_date.min()).delta.days+1
    first_day = str(df_cola.local_date.min()).split()[0]
    fh = ForecastingHorizon(pd.PeriodIndex(pd.date_range(first_day, periods=delta*24, freq="h")), is_relative=False).to_pandas()
    #print(fh)    
    df_tmp = pd.DataFrame()
    df_tmp['local_date'] = fh
    df_tmp['quantity'] = 0
    df_tmp['region'] = i
    #display(df_tmp)
    df_cola=pd.concat([df_cola, df_tmp], axis=0, join='outer')
    #display(df_cola)
    df_cola = df_cola.groupby(by=['region', 'local_date'], as_index=False).agg({'quantity':'sum'})

display(df_cola)



,region,local_date,quantity
0,Алтайский край,2021-02-01 00:00,0.0
1,Алтайский край,2021-02-01 01:00,0.0
2,Алтайский край,2021-02-01 02:00,0.0
3,Алтайский край,2021-02-01 03:00,0.0
4,Алтайский край,2021-02-01 04:00,0.0
...,...,...,...
5223,Ярославская область,2021-02-02 21:00,0.0
5224,Ярославская область,2021-02-02 22:00,0.0
5225,Ярославская область,2021-02-02 23:00,0.0
5226,Ярославская область,2021-02-04 18:00,1.0


In [811]:
df_cola.region.unique()

array(['Алтайский край', 'Астраханская область', 'Белгородская область',
       'Брянская область', 'Владимирская область',
       'Волгоградская область', 'Воронежская область',
       'Иркутская область', 'Кабардино-Балкарская Республика',
       'Калининградская область', 'Калужская область',
       'Костромская область', 'Краснодарский край', 'Красноярский край',
       'Курская область', 'Ленинградская область', 'Москва',
       'Московская область', 'Мурманская область',
       'Нижегородская область', 'Новосибирская область', 'Омская область',
       'Оренбургская область', 'Орловская область', 'Пермский край',
       'Приморский край', 'Республика Бурятия', 'Республика Коми',
       'Республика Крым', 'Республика Марий Эл',
       'Республика Саха (Якутия)', 'Республика Тыва',
       'Ростовская область', 'Рязанская область', 'Самарская область',
       'Санкт-Петербург', 'Саратовская область', 'Сахалинская область',
       'Свердловская область', 'Севастополь', 'Смоленская обл

Напишем функцию, которая будет на входе принимать название региона, обучать модель и выдавать результат

In [812]:
def func(region):
    mask = df_cola.region == region
    df = df_cola[mask]

    df = df[['local_date', 'quantity']]
    df = df.set_index('local_date')
    X = df[:len(df)-24] 
    y= df[len(df)-24:len(df)]
    #print('x=',X, len(y))
    #print('y=', )
    forecaster = AutoARIMA(sp=7, suppress_warnings=True)

    forecaster.fit(X)
    y_pred = forecaster.predict(fh=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,14,15,16,17,18,19,20,21,22,23,24])
    #print(y_pred)
    print('Метрики для оценки работы модели:')
    #Рассчитываем MAE
    print('MAE score: {:.3f} thou. $'.format(metrics.mean_absolute_error(y, y_pred)))
    #Рассчитываем RMSE
    print('RMSE score: {:.3f} thou. $'.format(np.sqrt(metrics.mean_squared_error(y, y_pred))))
    #Рассчитываем MAPE
    print('MAPE score: {:.3f} %'.format(metrics.mean_absolute_percentage_error(y, y_pred) * 100))
    #Рассчитываем коэффициент детерминации
    print('R2 score: {:.3f}'.format(metrics.r2_score(y, y_pred)))

    forecaster.fit(df) # обучение модели на всей выборке
    y_pred = forecaster.predict(fh=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,14,15,16,17,18,19,20,21,22,23,24])
    print(round(y_pred, 0))


    #return df
    

In [813]:
func('Московская область')

Метрики для оценки работы модели:
MAE score: 0.905 thou. $
RMSE score: 1.344 thou. $
MAPE score: 162373434878649824.000 %
R2 score: -0.033
                  quantity
2021-02-07 00:00       0.0
2021-02-07 01:00      -0.0
2021-02-07 02:00      -0.0
2021-02-07 03:00      -0.0
2021-02-07 04:00      -0.0
2021-02-07 05:00      -0.0
2021-02-07 06:00      -0.0
2021-02-07 07:00      -0.0
2021-02-07 08:00       0.0
2021-02-07 09:00       0.0
2021-02-07 10:00       1.0
2021-02-07 11:00       1.0
2021-02-07 12:00       1.0
2021-02-07 13:00       1.0
2021-02-07 14:00       1.0
2021-02-07 15:00       2.0
2021-02-07 16:00       2.0
2021-02-07 17:00       2.0
2021-02-07 18:00       1.0
2021-02-07 19:00       1.0
2021-02-07 20:00       1.0
2021-02-07 21:00       1.0
2021-02-07 22:00       1.0
2021-02-07 23:00       0.0


мае Помогает оценить абсолютную ошибку: насколько в среднем число в предсказании разошлось с реальным числом. Удобно интерпретировать. Измеряется в тех же единицах, что и целевой признак. Несильно искажается при наличии выбросов.
mape Помогает абстрагироваться от конкретных чисел и оценить абсолютную ошибку в процентах. Легко интерпретировать. Используется в задачах, где неизвестно, какое значение метрики считать приемлемым.

In [814]:
mask = df_cola.region == 'Московская область'
df_tmp = df_cola[mask]
df_tmp

,region,local_date,quantity
1841,Московская область,2021-02-01 00:00,0.0
1842,Московская область,2021-02-01 01:00,0.0
1843,Московская область,2021-02-01 02:00,0.0
1844,Московская область,2021-02-01 03:00,0.0
1845,Московская область,2021-02-01 04:00,0.0
...,...,...,...
1980,Московская область,2021-02-06 19:00,6.0
1981,Московская область,2021-02-06 20:00,1.0
1982,Московская область,2021-02-06 21:00,1.0
1983,Московская область,2021-02-06 22:00,2.0


In [815]:
df_tmp = df_tmp[['local_date', 'quantity']]
df_tmp = df_tmp.set_index('local_date')
df_tmp

,quantity
local_date,
2021-02-01 00:00,0.0
2021-02-01 01:00,0.0
2021-02-01 02:00,0.0
2021-02-01 03:00,0.0
2021-02-01 04:00,0.0
...,...
2021-02-06 19:00,6.0
2021-02-06 20:00,1.0
2021-02-06 21:00,1.0


In [816]:
X = df_tmp[:120]
y= df_tmp[120:144]
print(X)
print(len(y))

                  quantity
local_date                
2021-02-01 00:00       0.0
2021-02-01 01:00       0.0
2021-02-01 02:00       0.0
2021-02-01 03:00       0.0
2021-02-01 04:00       0.0
...                    ...
2021-02-05 19:00       0.0
2021-02-05 20:00       1.0
2021-02-05 21:00       0.0
2021-02-05 22:00       2.0
2021-02-05 23:00       0.0

[120 rows x 1 columns]
24


In [817]:
forecaster = AutoARIMA(sp=7, suppress_warnings=True)

forecaster.fit(X)
y_pred = forecaster.predict(fh=[0,1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,14,15,16,17,18,19,20,21,22,23])
print(y_pred)

                  quantity
2021-02-05 23:00  0.613804
2021-02-06 00:00  0.808111
2021-02-06 01:00  0.665157
2021-02-06 02:00  0.567046
2021-02-06 03:00  0.639727
2021-02-06 04:00  0.600876
2021-02-06 05:00  0.601720
2021-02-06 06:00  0.602629
2021-02-06 07:00  0.597366
2021-02-06 08:00  0.597307
2021-02-06 09:00  0.596142
2021-02-06 10:00  0.595289
2021-02-06 11:00  0.594929
2021-02-06 12:00  0.594527
2021-02-06 13:00  0.594286
2021-02-06 14:00  0.594118
2021-02-06 15:00  0.593989
2021-02-06 16:00  0.593904
2021-02-06 17:00  0.593841
2021-02-06 18:00  0.593797
2021-02-06 19:00  0.593767
2021-02-06 20:00  0.593745
2021-02-06 21:00  0.593729
2021-02-06 22:00  0.593718


In [818]:
#Рассчитываем MAE
print('MAE score: {:.3f} thou. $'.format(metrics.mean_absolute_error(y, y_pred)))
#Рассчитываем RMSE
print('RMSE score: {:.3f} thou. $'.format(np.sqrt(metrics.mean_squared_error(y, y_pred))))
#Рассчитываем MAPE
print('MAPE score: {:.3f} %'.format(metrics.mean_absolute_percentage_error(y, y_pred) * 100))
#Рассчитываем коэффициент детерминации
print('R2 score: {:.3f}'.format(metrics.r2_score(y, y_pred)))

MAE score: 0.905 thou. $
RMSE score: 1.344 thou. $
MAPE score: 162725009757520128.000 %
R2 score: -0.034


R2 - Можно сравнивать модели, обученные на разных признаках. Легко оценить качество модели: измеряется от  до 1. Удовлетворительным показателем считается показатель выше 0.5.

In [819]:
forecaster = AutoARIMA(sp=7, suppress_warnings=True)

forecaster.fit(df_tmp)
y_pred = forecaster.predict(fh=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,14,15,16,17,18,19,20,21,22,23])

In [820]:
print(round(y_pred, 0))

                  quantity
2021-02-07 00:00       0.0
2021-02-07 01:00      -0.0
2021-02-07 02:00      -0.0
2021-02-07 03:00      -0.0
2021-02-07 04:00      -0.0
2021-02-07 05:00      -0.0
2021-02-07 06:00      -0.0
2021-02-07 07:00      -0.0
2021-02-07 08:00       0.0
2021-02-07 09:00       0.0
2021-02-07 10:00       1.0
2021-02-07 11:00       1.0
2021-02-07 12:00       1.0
2021-02-07 13:00       1.0
2021-02-07 14:00       1.0
2021-02-07 15:00       2.0
2021-02-07 16:00       2.0
2021-02-07 17:00       2.0
2021-02-07 18:00       1.0
2021-02-07 19:00       1.0
2021-02-07 20:00       1.0
2021-02-07 21:00       1.0
2021-02-07 22:00       1.0
